## Carga de datos

In [1]:
library(dplyr)
library(abind)
library(loadeR)
library(gridExtra)
library(loadeR.2nc)
library(visualizeR)
library(transformeR)
library(RColorBrewer)
library(easyVerification)
library(climate4R.indices)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: transformeR




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

Loading required package: SpecsVerification


Attaching package: ‘easyVerification’


The following object is masked from ‘package:SpecsVerification’:

    EnsCorr


climate4R.indices version 0.3.1 (2023-06-22) is loaded

Use 'indexShow()' for an overview of the available climate indices and circIndexShow() for the circulation indices.

NOTE: use package climate4R.climdex to calculate ETCCDI indices.


Attaching package: ‘climate4R.indices’


The following object is masked from ‘package:transformeR’:

    lambWT




In [2]:
# Color
color = colorRampPalette(rev(brewer.pal(n = 9, "RdYlBu")))

# Datos
pr_seas5_1 = readRDS('../../data/npd_s_w/pr_npds_seas5_1.rds')
pr_seas5_1_bc = readRDS('../../data/npd_s_w/pr_npds_seas5_1_bc.rds')
pr_era5_1 = readRDS('../../data/npd_s_w/pr_npds_era5_1.rds')

index_raw_masked = readRDS('../../data/npd_s_w/indices_npd_s_w/npd_s_seas5_raw.rds')
index_obs_masked = readRDS('../../data/npd_s_w/indices_npd_s_w/npd_s_era5.rds')
index_bc_masked = readRDS('../../data/npd_s_w/indices_npd_s_w/npd_s_seas5_bc.rds')

## Representación del valor del índice

In [7]:
value_obs = spatialPlot(climatology(index_obs_masked, by.member = FALSE),
                        backdrop.theme = "countries",
                        main = "npd_s (obs)",
                        col.regions = color,
                        at = seq(10, 200, 1)) %>% suppressMessages %>% suppressWarnings

value_raw = spatialPlot(climatology(index_raw_masked, by.member = FALSE),
                        backdrop.theme = "countries",
                        main = "npd_s (raw)",
                        col.regions = color,
                        at = seq(10, 200, 1)) %>% suppressMessages %>% suppressWarnings

value_bc = spatialPlot(climatology(index_bc_masked, by.member = FALSE),
                       backdrop.theme = "countries",
                       main = "npd_s (bc)",
                       col.regions = color,
                       at = seq(10, 200, 1)) %>% suppressMessages %>% suppressWarnings

png("npd_s.png", width = 2000, height = 1000, res = 150)
grid.arrange(value_obs, value_raw, value_bc, ncol = 3)
dev.off()

pdf 
  2

In [18]:
# Bias pr raw
ref_pr = climatology(pr_era5_1) %>% suppressMessages %>% suppressWarnings
diff_pr = climatology(pr_seas5_1, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
bias_pr = gridArithmetics(diff_pr, ref_pr, operator = "-")
b_pr = spatialPlot(bias_pr, backdrop.theme = "countries", main = "Bias pr (raw)", col.regions = color, at = seq(-2, 2, 0.1))

# Bias tmax bc
diff_pr_bc = climatology(pr_seas5_1_bc, by.member = FALSE) %>% suppressMessages %>% suppressWarnings
bias_pr_bc = gridArithmetics(diff_pr_bc, ref_pr, operator = "-")
b_pr_bc = spatialPlot(bias_pr_bc, backdrop.theme = "countries", main = "Bias pr (bc)", col.regions = color, at = seq(-0.5, 0.5, 0.01))

In [19]:
png("bias_pr_npd_s.png", width = 2000, height = 1000, res = 150)
grid.arrange(b_pr, b_pr_bc, ncol = 2)
dev.off()

pdf 
  2

## Bias ndays

### obs y raw

In [20]:
# Calculo del bias del ndays
bias_raw = veriApply(verifun = "EnsMe", 
                     fcst = index_raw_masked$Data, 
                     obs = index_obs_masked$Data,
                     ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_raw_grid = easyVeri2grid(easyVeri.mat = bias_raw, obs.grid = index_obs_masked, verifun = "EnsMe")

# Representación
bias_raw_plot = spatialPlot(climatology(bias_raw_grid),
                            backdrop.theme = "countries",
                            col.regions = color,
                            main = "Bias (raw)",
                            at = seq(-100, 100, 1)) %>% suppressMessages %>% suppressWarnings

### obs y bc

In [22]:
# Calculo del bias del ndays
bias_bc = veriApply(verifun = "EnsMe", 
                    fcst = index_bc_masked$Data, 
                    obs = index_obs_masked$Data, 
                    ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
bias_bc_grid = easyVeri2grid(easyVeri.mat = bias_bc, obs.grid = index_obs_masked, verifun = "EnsMe")

# Representación
bias_bc_plot = spatialPlot(climatology(bias_bc_grid),
                           backdrop.theme = "countries",
                           col.regions = color,
                           main = "Bias (bc)",
                           at = seq(-1, 1, 0.1)) %>% suppressMessages %>% suppressWarnings

## Corr ndays

### test de hipótesis para puntos significativos

In [11]:
# Función para calcular correlación de Pearson y valores p entre datos de modelo y observaciones en una grilla espacial
# Además, identifica y marca los puntos con correlación estadísticamente significativa según un umbral de p-valor
#
# Args:
#   model_data: objeto con datos del modelo, estructura esperada con dimensión [miembros, tiempo, latitud, longitud]
#   obs_data: objeto con datos observacionales, estructura con dimensión [tiempo, latitud, longitud]
#   ref_grid: objeto referencia con metadatos espaciales y temporales para construir grillas (xyCoords, Variable, Dates)
#   threshold: umbral para marcar significancia estadística (p-valor), default 0.05
#
# Returns:
#   Lista con:
#     - cor: matriz de correlaciones [lat x lon]
#     - pval: matriz de valores p [lat x lon]
#     - pval_grid: objeto tipo "grid" con valores p y metadatos
#     - pts: lista de objetos para graficar puntos de significancia (stippling)

calc_cor_pval_grid = function(model_data, obs_data, ref_grid, threshold = 0.05) {
    
    # Calcular la media del ensamble para cada punto [tiempo, lat, lon]
    ens_mean = apply(model_data$Data, c(2, 3, 4), mean, na.rm = TRUE)
    
    # Dimensiones espaciales (latitud y longitud)
    lat_n = dim(ens_mean)[2]
    lon_n = dim(ens_mean)[3]
  
    # Inicializar matrices vacías para almacenar correlaciones y p-valores
    cor_array = matrix(NA, nrow = lat_n, ncol = lon_n)
    pval_array = matrix(NA, nrow = lat_n, ncol = lon_n)
    
    # Iterar sobre cada punto espacial
    for (i in 1:lat_n) {
        for (j in 1:lon_n) {
            
            # Extraer series temporales de modelo y observaciones para la celda actual
            pred_series = ens_mean[, i, j]
            obs_series = obs_data$Data[, i, j]
      
            # Filtrar índices con datos completos (no NA)
            valid_idx = complete.cases(pred_series, obs_series)
            
            # Solo calcular correlación si hay suficientes datos (mínimo 10)
            if (sum(valid_idx) >= 10) {
                test = cor.test(pred_series[valid_idx], obs_series[valid_idx], method = "pearson")
                cor_array[i, j] = test$estimate  # Coeficiente de correlación
                pval_array[i, j] = test$p.value  # Valor p de la prueba
            }
        }
    }
  
    # Construir un objeto "grid" para los valores p, con metadatos espaciales y temporales
    pval_grid = list()
    pval_grid$Data = pval_array
    attr(pval_grid$Data, "dimensions") = c("lat", "lon")
    pval_grid$xyCoords = ref_grid$xyCoords
    pval_grid$Variable = ref_grid$Variable
    pval_grid$Dates = ref_grid$Dates
    class(pval_grid) = "grid"

    pval_grid$Variable$varName = "p-values"
    attr(pval_grid$Variable, "description") = "Mapa de p-valores"
    attr(pval_grid$Variable, "units") = ""
    attr(pval_grid$Variable, "longname") = "p-values"
    
    # Crear objetos para graficar puntos de significancia estadística (stippling)
    pts = map.stippling(climatology(pval_grid), 
                        threshold = threshold, 
                        condition = "LT", 
                        pch = 19, col = "black", cex = 0.5) %>% suppressMessages() %>% suppressWarnings()
    
    # Devolver lista con resultados y objetos para plot
    return(list(cor = cor_array, pval = pval_array, pval_grid = pval_grid, pts = pts))
}

### obs y raw

In [12]:
# Datos raw tmax
test_cor_tmax_raw = calc_cor_pval_grid(index_raw_masked, index_obs_masked, index_raw_masked)

# Calculo del bias del ndays
corr_raw = veriApply(verifun = "EnsCorr", 
                     fcst = index_raw_masked$Data, 
                     obs = index_obs_masked$Data, 
                     ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Reconstrucción del grid
corr_raw_grid = easyVeri2grid(easyVeri.mat = corr_raw, obs.grid = index_obs_masked, verifun = "EnsCorr")

# Representación
corr_raw_plot = spatialPlot(climatology(corr_raw_grid),
                            backdrop.theme = "countries",
                            sp.layout = list(test_cor_tmax_raw$pts),
                            col.regions = color,
                            main = "Corr (raw)",
                            at = seq(-1, 1, 0.1)) %>% suppressMessages %>% suppressWarnings

In [13]:
png("bias_corr_npd_s.png", width = 2000, height = 1000, res = 150)
grid.arrange(bias_raw_plot, corr_raw_plot, ncol = 2)
dev.off()

pdf 
  2

## ROCSS

### Funciones auxiliares

In [14]:
## Función para realizar bootstrap y calcular ROCSS
##
## Esta función calcula el ROC Skill Score (ROCSS) mediante un enfoque bootstrap,
## generando múltiples muestras re-muestreadas con reemplazo sobre la dimensión temporal.
##
## Argumentos:
## - nd_modelo: objeto con los pronósticos originales (formato esperado: [member, time, lat, lon])
## - nd_obs: objeto con las observaciones correspondientes (formato: [time, lat, lon])
## - n_boot: número de muestras bootstrap a generar (por defecto: 1000)
##
## Valor:
## - Lista de 3 arrays (uno por tercil), cada uno con dimensiones [n_boot, lat, lon],
##   que contiene los valores bootstrap del ROCSS para cada tercil.

rocss_bootstrap = function(nd_modelo, nd_obs, n_boot = 1000) {
    
    # Obtener dimensiones del array de pronóstico
    dims = dim(nd_modelo$Data)
    
    # Inicializar lista para guardar ROCSS bootstrap para los tres terciles
    rocss_bootstrap = vector("list", length = 3)
    
    # Extraer observaciones
    obs = nd_obs$Data
    
    # Inicializar arrays vacíos para cada tercil: [n_boot, lat, lon]
    for (i in 1:3) {
        rocss_bootstrap[[i]] = array(NA, dim = c(n_boot, dims[3], dims[4]))
    }
    
    # Bucle principal de bootstrap
    for (b in 1:n_boot) {
        
        # Muestreo con reemplazo sobre la dimensión temporal (índice de tiempo)
        time_idx = sample(1:dims[2], size = dims[2], replace = TRUE)
    
        # Crear pronóstico re-muestreado: [member, time*, sdate, lat, lon]
        fcst_boot = nd_modelo$Data[, time_idx, , , drop = FALSE]
    
        # Calcular ROCSS con veriApply usando la función EnsRocss
        boot_result = veriApply(
            verifun = "EnsRocss",
            fcst = fcst_boot,
            obs = obs,
            prob = c(1/3, 2/3),
            ensdim = 1,   # Dimensión de los miembros del ensamble
            tdim = 2      # Dimensión temporal
        )
    
        # Guardar el resultado para cada tercil en la lista correspondiente
        for (i in 1:3) {
            rocss_bootstrap[[i]][b, , ] = boot_result[[i]]
        }
    }
    
    # Devolver lista con resultados bootstrap por tercil
    return(rocss_bootstrap)
}

In [15]:
## Función para generar capas de "stippling" que indican puntos con ROCSS significativamente mayores que un percentil dado
##
## Parámetros:
## mg               : objeto con datos ROCSS con dimensiones [tercil, member, time, lat, lon]
## rocss_bootstrap  : lista de arrays bootstrap con dimensiones [n_boot, lat, lon] para cada tercil
## ref_grid         : grid de referencia que contiene Dates y xyCoords (coordenadas XY)
## threshold        : percentil del bootstrap para determinar significancia (default 0.95)
##
## Retorna:
## lista de objetos "sp.points" para cada tercil con puntos significativos (stippling)

get_rocss_stippling_layers = function(mg, rocss_bootstrap, ref_grid, threshold = 0.95) {
	
	rocss_orig = mg$Data
	n_terciles = dim(rocss_orig)[1]
	n_lat = dim(rocss_orig)[4]
	n_lon = dim(rocss_orig)[5]
	
	xyCoords = ref_grid$xyCoords
	Dates = ref_grid$Dates
	
	stippling_list = vector("list", n_terciles)
	
	for (t in 1:n_terciles) {
		signif_mask = matrix(NA, nrow = n_lat, ncol = n_lon)
		
		for (i in 1:n_lat) {
			for (j in 1:n_lon) {
				boot_vals = rocss_bootstrap[[t]][, i, j]
				val_orig = rocss_orig[t, 1, 1, i, j]
				if (all(is.na(boot_vals)) || is.na(val_orig)) next
				q95 = quantile(boot_vals, threshold, na.rm = TRUE)
				signif_mask[i, j] = val_orig > q95
			}
		}
		
		# Construcción manual del objeto grid para la capa de significancia
		pval_grid = list()
		pval_grid$Data = signif_mask
		attr(pval_grid$Data, "dimensions") = c("lat", "lon")
		pval_grid$Dates = Dates
		pval_grid$xyCoords = xyCoords
		pval_grid$Variable = list(varName = paste0("significance_tercil_", t))
		class(pval_grid) = "grid"
		
		# Genera puntos de significancia con map.stippling (pch=19, color negro, tamaño 0.5)
		stippling_list[[t]] = suppressWarnings(
			suppressMessages(
				map.stippling(climatology(pval_grid), threshold = 0.5, condition = "GT",
				              pch = 19, col = "black", cex = 0.5)
			)
		)
	}
	
	# Elimina entradas NULL (si algún tercil no tiene puntos significativos)
	stippling_list = Filter(Negate(is.null), stippling_list)
	
	# Filtra para quedarse solo con listas que son puntos espaciales (sp.points)
	stippling_list = Filter(function(x) {
		is.list(x) && length(x) > 1 && x[[1]] == "sp.points"
	}, stippling_list)
	
	return(stippling_list)
}

### obs y raw

In [16]:
# Calculo del ROCSS para cada leadtime del modelo
rocss_raw = veriApply(verifun = "EnsRocss", 
                      fcst = index_raw_masked$Data, 
                      obs = index_obs_masked$Data,
                      prob = c(1/3, 2/3),
                      ensdim = 1, tdim = 2) %>% suppressMessages %>% suppressWarnings

# Multigrid para representar los tres percentiles
mg_rocss_raw = makeMultiGrid(lapply(rocss_raw[1:3], "easyVeri2grid", index_obs_masked))

set.seed(123)  # Reproducibilidad

# Aplico función de bootstraping
rocss_bootstrap_raw = rocss_bootstrap(index_raw_masked, index_obs_masked)

# Aplico función de máscara significativa
pts_layers_raw = get_rocss_stippling_layers(mg_rocss_raw, rocss_bootstrap_raw, index_raw_masked)

In [17]:
rocss_raw = spatialPlot(climatology(mg_rocss_raw),
                        backdrop.theme = "countries",
                        names.attr = c("Lower tercile", "Middle tercile", "Upper tercile"),
                        layout = c(3,1),
                        col.regions = color,
                        at = seq(-1, 1, 0.05),
                        sp.layout = list(pts_layers_raw),
                        main = "ROCSS (raw)") %>% suppressMessages %>% suppressWarnings

In [18]:
png("rocss_npd_s.png", width = 2000, height = 1000, res = 150)
grid.arrange(rocss_raw, ncol = 1)
dev.off()

pdf 
  2